In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
!pip install wandb

In [4]:
import wandb
wandb.login(key="e7f4198ca97d01e59270148ce3f1ceda815823e6")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
!pip install --upgrade pip
!git clone https://github.com/huggingface/instruction-tuned-sd.git
%cd instruction-tuned-sd/
!pwd
!pip install -r requirements.txt
!apt-get update
!apt-get install git-lfs
!pip install --upgrade pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Cloning into 'instruction-tuned-sd'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 187 (delta 14), reused 12 (delta 12), pack-reused 168 (from 1)
Receiving objects: 100% (187/187), 58.34 KiB | 8.33 MiB/s, done.
Resolving deltas: 100% (102/102), done.
/kaggle/working/instruction-tuned-sd
/kaggle/working/instruction-tuned-sd
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 35.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
   

In [10]:
!accelerate launch --mixed_precision="fp16" /workspace/instruction-tuned-sd/finetune_instruct_pix2pix.py \
  --pretrained_model_name_or_path='timbrooks/instruct-pix2pix' \
  --dataset_name='maksymveremchuk/steel2' \
  --original_image_column="input_image" \
  --edit_prompt_column="edit_prompt" \
  --edited_image_column="output_image" \
  --use_ema \
  --resolution=512 \
  --random_flip \
  --train_batch_size=2 --gradient_accumulation_steps=8 \
  --max_train_steps=2000 \
  --max_train_samples=300 \
  --checkpointing_steps=150 --checkpoints_total_limit=1 \
  --learning_rate=5e-05 --lr_warmup_steps=0 \
  --mixed_precision=fp16 \
  --allow_tf32 \
  --val_image_url="https://i.pinimg.com/564x/ed/6f/c2/ed6fc28d25da28c73e45ed762a26bbd3.jpg" \
  --validation_prompt="Transform an image into a black and white drawing that captures only the main features with no background" \
  --seed=42 \
  --output_dir='petsimg2img_sd' \
  --report_to=wandb\
  --push_to_hub

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
scheduler/scheduler_config.json: 100%|█████████| 569/569 [00:00<00:00, 3.17MB/s]
{'rescale_betas_zero_snr', 'variance_type', 'timestep_spacing', 'clip_sample_range'} was not found in config. Values will be initialized to default values.
tokenizer/tokenizer_config.json: 100%|█████████| 806/806 [00:00<00:00, 5.34MB/s]
tokenizer/vocab.json: 100%|████████████████| 1.06M/1.06M [00:00<00:00, 21.2MB/s]
tokenizer/merges.txt: 100%|██████████████████| 525k/525k [00:00<00:00, 48.7MB/s]
tokenizer/special_tokens_map.json: 100%|███████| 472/472 [00:00<00:00, 3.20MB/s]
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This be